In [1]:
import numpy as np
import random
import tensorflow as tf
import pandas as pd
import pickle
from keras_facenet import FaceNet

from tensorflow.keras import datasets, layers, models

from keras.models import load_model
from skimage.io import imread
from keras.preprocessing import image
from skimage.transform import resize
from matplotlib import pyplot as plt
from keras.optimizers import RMSprop, SGD, Adam
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda
import gc

import os

c:\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a fut

In [2]:
with open('data/train_img_embeddings.pkl', 'rb') as f:
       train_embeddings = pickle.load(f)
print(len(train_embeddings))

2318


In [3]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    '''
    Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

def compute_accuracy(y_true, y_pred):
    # Compute classification accuracy with a fixed threshold on distances.
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)

def accuracy(y_true, y_pred):
    # Compute classification accuracy with a fixed threshold on distances.
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [4]:
def pairs_set(input_pairs):
    '''
    Create positive pairs according to input .csv file.
    Negative pairs are generated by randomly picking people from different families.
    
    Arguments:
    input_pairs -- pandas DataFrame with positive pairs paths
    
    Returns:
    train_pairs -- array of pairs of embeddings
    labels -- labels for each pair, 1 for positive(in blood relation) and 0 for negative
    '''
    n = len(input_pairs.index)
    train_pairs = []
    labels = []
    errors = 0
    for idx, row in input_pairs.iterrows():
        try:
            #Add positive pair
            new_pairs = make_pairs(row['p1'], row['p2'])
            train_pairs += new_pairs
            labels += [1] * len(new_pairs)
            
            #Add negative pair
            rnd_idx = np.random.randint(n)
            while(row['p1'][:5] == input_pairs.iloc[rnd_idx][1][:5]):
                rnd_idx = np.random.randint(n)
            
            new_pairs = make_pairs(row['p1'], input_pairs.iloc[rnd_idx][1])
            train_pairs += new_pairs
            labels += [0] * len(new_pairs)
            
        except KeyError:
            errors += 1
    print(f'\nThere are {errors} key errors of {len(train_rltshps)} relationships.')
    return np.array(train_pairs), np.array(labels)

In [5]:
def make_pairs(p1, p2):
    '''
    Create pair of embeddings.
    
    Arguments:
    p1, p2 -- paths to persons' images directories (familyID/personID)
    
    Returns:
    pairs -- array of image pairs, pairing is alligned to smaller number of images
    '''
    pairs = []
    img_path1 = p1.replace('/', '\\')
    img_path2 = p2.replace('/', '\\')
    
    dir1 = train_embeddings[img_path1]
    dir2 = train_embeddings[img_path2]
    n = min(len(dir1), len(dir2))
    
    for i in range(n):
        pairs.append([dir1[i], dir2[i]])
    return pairs

In [6]:
def create_base_network(input_shape):
    '''
    Create deep Keras model.
    
    Arguments:
    input_shape -- shape of the input layer
    
    Returns:
    Model -- Keras model
    '''
    input = Input(shape=input_shape)
    x = Dense(input.shape[1].value // 2, activation = 'relu')(input)
    x = Dropout(0.1)(x)
    x = Dense(input.shape[1].value // 4, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(input.shape[1].value // 8, activation='relu')(x)
    return Model(input, x)

In [7]:
# Read relatives' pairs
train_rltshps = pd.read_csv("data/train_relationships.csv")
train_rltshps.head()

,p1,p2
0,F0002/MID1,F0002/MID3
1,F0002/MID2,F0002/MID3
2,F0005/MID1,F0005/MID2
3,F0005/MID3,F0005/MID2
4,F0009/MID1,F0009/MID4


In [8]:
# Shuffle rows in pandas DataFrame
train_rltshps = train_rltshps.sample(frac=1).reset_index(drop=True)
train_rltshps.info

<bound method DataFrame.info of                p1           p2
0      F0881/MID1   F0881/MID4
1      F0842/MID3   F0842/MID4
2     F0357/MID13  F0357/MID10
3      F0081/MID1   F0081/MID6
4      F0734/MID4   F0734/MID5
...           ...          ...
3593   F0051/MID5   F0051/MID4
3594   F0825/MID5   F0825/MID4
3595   F0805/MID3   F0805/MID6
3596   F0818/MID8   F0818/MID9
3597   F0696/MID5   F0696/MID6

[3598 rows x 2 columns]>

In [9]:
# Create training and validation sets
val_rltshps = train_rltshps.iloc[3300:]
train_rltshps = train_rltshps.iloc[:3300]
print(train_rltshps.shape)
print(val_rltshps.shape)

(3300, 2)
(298, 2)


In [10]:
train_pairs, train_labels = pairs_set(train_rltshps)
val_pairs, val_labels = pairs_set(val_rltshps)


There are 333 key errors of 3300 relationships.

There are 32 key errors of 3300 relationships.


In [11]:
print(train_pairs.shape)
print(val_pairs.shape)

(19990, 2, 512)
(1856, 2, 512)


In [12]:
# Input has 512 embeddings
m_shape = (512,)
base_network = create_base_network(m_shape)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
# Creation of Siamese network
input1 = Input(shape=m_shape)
input2 = Input(shape=m_shape)
processed1 = base_network(input1)
processed2 = base_network(input2)

In [14]:
distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed1, processed2])

model = Model([input1, input2], distance)

In [15]:
rms = Adam()
model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])

In [16]:
epochs = 200
model.fit([train_pairs[:, 0], train_pairs[:, 1]], train_labels,
              batch_size=128, epochs=epochs, validation_data=([val_pairs[:, 0], val_pairs[:, 1]], val_labels))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 19990 samples, validate on 1856 samples
Epoch 1/200
19990/19990 [==============================] - 2s 80us/step - loss: 0.2097 - accuracy: 0.6711 - val_loss: 0.1949 - val_accuracy: 0.7010
Epoch 2/200
19990/19990 [==============================] - 1s 68us/step - loss: 0.1811 - accuracy: 0.7421 - val_loss: 0.1880 - val_accuracy: 0.7123
Epoch 3/200
19990/19990 [==============================] - 1s 73us/step - loss: 0.1708 - accuracy: 0.7640 - val_loss: 0.1838 - val_accuracy: 0.7128
Epoch 4/200
19990/19990 [==============================] - 2s 76us/step - loss: 0.1603 - accuracy: 0.7877 - val_loss: 0.1793 - val_accuracy: 0.7268
Epoch 5/200
19990/19990 [==============================] - 2s 75us/step - loss: 0.1512 - accuracy: 0.8074 - val_loss: 0.1750 - val_accuracy: 0.7446
Epoch 6/200
19990/19990 [==============================] - 1s 74us/step - loss: 0.1441 - accuracy: 0.8207 - val_loss

19990/19990 [==============================] - 1s 57us/step - loss: 0.0622 - accuracy: 0.9557 - val_loss: 0.1274 - val_accuracy: 0.8314
Epoch 53/200
19990/19990 [==============================] - 1s 57us/step - loss: 0.0624 - accuracy: 0.9541 - val_loss: 0.1274 - val_accuracy: 0.8319
Epoch 54/200
19990/19990 [==============================] - 1s 57us/step - loss: 0.0616 - accuracy: 0.9550 - val_loss: 0.1252 - val_accuracy: 0.8314
Epoch 55/200
19990/19990 [==============================] - 1s 65us/step - loss: 0.0611 - accuracy: 0.9543 - val_loss: 0.1245 - val_accuracy: 0.8270
Epoch 56/200
19990/19990 [==============================] - 2s 78us/step - loss: 0.0618 - accuracy: 0.9536 - val_loss: 0.1261 - val_accuracy: 0.8276
Epoch 57/200
19990/19990 [==============================] - 1s 68us/step - loss: 0.0599 - accuracy: 0.9577 - val_loss: 0.1254 - val_accuracy: 0.8281
Epoch 58/200
19990/19990 [==============================] - 1s 56us/step - loss: 0.0600 - accuracy: 0.9574 - val_loss: 

19990/19990 [==============================] - 1s 68us/step - loss: 0.0478 - accuracy: 0.9665 - val_loss: 0.1138 - val_accuracy: 0.8438
Epoch 108/200
19990/19990 [==============================] - 1s 68us/step - loss: 0.0484 - accuracy: 0.9650 - val_loss: 0.1136 - val_accuracy: 0.8475
Epoch 109/200
19990/19990 [==============================] - 1s 67us/step - loss: 0.0474 - accuracy: 0.9671 - val_loss: 0.1134 - val_accuracy: 0.8513
Epoch 110/200
19990/19990 [==============================] - 1s 65us/step - loss: 0.0477 - accuracy: 0.9638 - val_loss: 0.1122 - val_accuracy: 0.8497
Epoch 111/200
19990/19990 [==============================] - 1s 64us/step - loss: 0.0478 - accuracy: 0.9650 - val_loss: 0.1114 - val_accuracy: 0.8475
Epoch 112/200
19990/19990 [==============================] - 2s 80us/step - loss: 0.0481 - accuracy: 0.9651 - val_loss: 0.1119 - val_accuracy: 0.8524
Epoch 113/200
19990/19990 [==============================] - 2s 80us/step - loss: 0.0479 - accuracy: 0.9649 - val_

19990/19990 [==============================] - 1s 65us/step - loss: 0.0428 - accuracy: 0.9677 - val_loss: 0.1143 - val_accuracy: 0.8508
Epoch 162/200
19990/19990 [==============================] - 1s 66us/step - loss: 0.0426 - accuracy: 0.9664 - val_loss: 0.1108 - val_accuracy: 0.8540
Epoch 163/200
19990/19990 [==============================] - 1s 68us/step - loss: 0.0414 - accuracy: 0.9698 - val_loss: 0.1090 - val_accuracy: 0.8551
Epoch 164/200
19990/19990 [==============================] - 1s 69us/step - loss: 0.0414 - accuracy: 0.9679 - val_loss: 0.1123 - val_accuracy: 0.8502
Epoch 165/200
19990/19990 [==============================] - 1s 69us/step - loss: 0.0422 - accuracy: 0.9663 - val_loss: 0.1101 - val_accuracy: 0.8534
Epoch 166/200
19990/19990 [==============================] - 1s 69us/step - loss: 0.0426 - accuracy: 0.9664 - val_loss: 0.1117 - val_accuracy: 0.8513
Epoch 167/200
19990/19990 [==============================] - 1s 67us/step - loss: 0.0430 - accuracy: 0.9652 - val_

In [17]:
# Compute test accuracy
y_pred = model.predict([val_pairs[:, 0], val_pairs[:, 1]])
te_acc = compute_accuracy(val_labels, y_pred)
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

* Accuracy on test set: 84.97%
